In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tensorflow import keras
from tensorflow.keras import layers
import cv2
import shutil
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import tensorflow as tf
from tensorflow.keras.optimizers import Adam


In [2]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split

In [3]:
# Listing all files
os.listdir('/kaggle/input/camp-qmind-hackathon-2023-A')

['sample_submission.csv', 'train.csv', 'test.csv']

In [4]:
train_df = pd.read_csv('/kaggle/input/camp-qmind-hackathon-2023-A/train.csv')
#cleaning train data
train_df = train_df.replace("05-Mar", "3-5")
train_df = train_df.replace("14-Oct", "10-14")
train_df = train_df.replace("08-Jun", "6-8")
train_df = train_df.replace("14-Dec", "12-14")
train_df = train_df.replace("09-May", "5-9")
train_df = train_df.replace("11-Sep", "9-11")

train_df = train_df.drop(train_df[train_df['Node-caps'] == '?'].index)
train_df = train_df.drop(train_df[train_df['Breast-quad'] == '?'].index)


#train_df.head()  # An overview of the training set

In [5]:
#cleaning test data
validate_df = pd.read_csv('/kaggle/input/camp-qmind-hackathon-2023-A/test.csv')



validate_df = validate_df.replace("05-Mar", "3-5")
validate_df = validate_df.replace("14-Oct", "10-14")
validate_df = validate_df.replace("08-Jun", "6-8")
validate_df = validate_df.replace("09-May", "5-9")
validate_df = validate_df.replace("11-Sep", "9-11")

validate_df = validate_df.drop(validate_df[validate_df['Node-caps'] == '?'].index)






In [6]:

zero_temp_df = train_df.loc[(train_df["Class"]==0)]
one_temp_df = train_df.loc[(train_df["Class"]==1)]


zero_temp_df = zero_temp_df.sample(frac=0.416, random_state=31).reset_index(drop=True)


train_df = pd.concat([one_temp_df, zero_temp_df], ignore_index = True)


print(train_df.head())

labels = train_df["Class"]
train_df = train_df.drop(columns = ["Class"])



   ID    Age Menopause Tumor-size Inv-nodes Node-caps  Deg-malig Breast  \
0  63  50-59      ge40      30-34       3-5        no          3   left   
1  64  60-69      ge40      30-34       3-5       yes          2   left   
2  65  30-39   premeno        0-4       0-2        no          2  right   
3  73  60-69      ge40      45-49       0-2        no          1  right   
4  81  40-49   premeno      25-29       0-2        no          2  right   

  Breast-quad Irradiat  Class  
0    left_low       no      1  
1     central      yes      1  
2     central       no      1  
3    right_up      yes      1  
4    left_low       no      1  


In [7]:
#one hot encoding
categorical_col = ["Age", "Menopause", "Tumor-size", "Inv-nodes", "Node-caps", "Breast", "Breast-quad", "Irradiat"]
train_encoder = OneHotEncoder()
#create separate DF with one-hot-encoded features
train_encoded_features = train_encoder.fit_transform(train_df[categorical_col]).toarray()


validate_encoder = OneHotEncoder()
#create separate DF with one-hot-encoded features
validate_encoded_features = validate_encoder.fit_transform(validate_df[categorical_col]).toarray()


In [8]:
numerical_col = ["Deg-malig"]
train_scaler = StandardScaler()
train_scaled_numerical_features = train_scaler.fit_transform(train_df[numerical_col])

validate_scaler = StandardScaler()
validate_scaled_numerical_features = validate_scaler.fit_transform(validate_df[numerical_col])

In [9]:
train_data = pd.concat([pd.DataFrame(train_encoded_features), pd.DataFrame(train_scaled_numerical_features)],  axis=1)


validate_data = pd.concat([pd.DataFrame(validate_encoded_features), pd.DataFrame(validate_scaled_numerical_features)],  axis=1)


In [10]:




x_train, x_test, y_train, y_test = train_test_split(train_data, labels, test_size = 0.2, random_state=31)





In [11]:
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

# Create Your Model

In [12]:
#pre-process data: 1-balance the data, 2-convert nominal data to numerical, 3-normalize data
model = tf.keras.models.Sequential([tf.keras.layers.Dense(128, input_shape = (x_train.shape[1],)),
                                    tf.keras.layers.Dense(64, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(32, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(1,activation='sigmoid')])

model.compile(optimizer = tf.optimizers.Adam(learning_rate = 0.001),
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

history = model.fit(x_train, y_train, batch_size=128, epochs=10, validation_split=0.15)

Epoch 1/10
1/1 [==============================] - 1s 1s/step - loss: 0.7117 - accuracy: 0.4659 - val_loss: 0.6255 - val_accuracy: 0.8125
Epoch 2/10
1/1 [==============================] - 0s 28ms/step - loss: 0.6791 - accuracy: 0.5455 - val_loss: 0.6149 - val_accuracy: 0.7500
Epoch 3/10
1/1 [==============================] - 0s 29ms/step - loss: 0.6514 - accuracy: 0.6705 - val_loss: 0.6081 - val_accuracy: 0.6875
Epoch 4/10
1/1 [==============================] - 0s 36ms/step - loss: 0.6279 - accuracy: 0.6932 - val_loss: 0.6034 - val_accuracy: 0.6875
Epoch 5/10
1/1 [==============================] - 0s 52ms/step - loss: 0.6080 - accuracy: 0.7386 - val_loss: 0.6006 - val_accuracy: 0.6875
Epoch 6/10
1/1 [==============================] - 0s 33ms/step - loss: 0.5916 - accuracy: 0.7500 - val_loss: 0.6001 - val_accuracy: 0.6875
Epoch 7/10
1/1 [==============================] - 0s 33ms/step - loss: 0.5777 - accuracy: 0.7500 - val_loss: 0.6010 - val_accuracy: 0.6875
Epoch 8/10
1/1 [=============

In [13]:
# define your model, train, and make predictions on the test set

# Preparing a CSV for Submission

In [14]:
# Reading in the test dataset
test_df = pd.read_csv('/kaggle/input/camp-qmind-hackathon-2023-A/test.csv')
test_ids = test_df['ID']
test_df.head() # Identical format to the train data, without a class label (your model will predict this)

,ID,Age,Menopause,Tumor-size,Inv-nodes,Node-caps,Deg-malig,Breast,Breast-quad,Irradiat
0,1,60-69,ge40,14-Oct,0-2,no,1,left,left_low,no
1,2,50-59,premeno,35-39,15-17,yes,3,right,right_up,no
2,3,60-69,ge40,30-34,0-2,no,3,left,left_low,no
3,4,50-59,lt40,20-24,0-2,?,1,left,left_up,no
4,5,60-69,ge40,15-19,0-2,no,1,right,left_up,no


In [15]:
# When you have the ids and predictions prepared, here is how you make a submission.csv
predicted_class = [0] * 58  #TODO: replace this array with your own predictions. These are placeholder predictions.

submission = {"ID": np.array(test_ids).astype(int), "Class": predicted_class}
submisson_df = pd.DataFrame(submission)

# you can sort the data frame by ID (you don't have to)
submisson_df = submisson_df.sort_values(by=['ID'])
print(submisson_df)
submisson_df.to_csv('/kaggle/working/submission.csv', index=False)

    ID  Class
0    1      0
1    2      0
2    3      0
3    4      0
4    5      0
5    6      0
6    7      0
7    8      0
8    9      0
9   10      0
10  11      0
11  12      0
12  13      0
13  14      0
14  15      0
15  16      0
16  17      0
17  18      0
18  19      0
19  20      0
20  21      0
21  22      0
22  23      0
23  24      0
24  25      0
25  26      0
26  27      0
27  28      0
28  29      0
29  30      0
30  31      0
31  32      0
32  33      0
33  34      0
34  35      0
35  36      0
36  37      0
37  38      0
38  39      0
39  40      0
40  41      0
41  42      0
42  43      0
43  44      0
44  45      0
45  46      0
46  47      0
47  48      0
48  49      0
49  50      0
50  51      0
51  52      0
52  53      0
53  54      0
54  55      0
55  56      0
56  57      0
57  58      0
